In [1]:
! pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

In [ ]:
# conectado ao drive através da interface gráfica
'''
from google.colab import drive
drive.mount('/content/drive')
'''

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
pagamentos_pedido_preprocessados = spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/pagamentos_pedido_preprocessados_parquet')
join_itens_pedido = spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/join_itens_pedido_parquet')

In [5]:
pagamentos_pedido_preprocessados.show(truncate=False)
join_itens_pedido.show(truncate=False)

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333            |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664          |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664          |
|ba78997921bbcdc

In [6]:
pagamentos_pedido_preprocessados.corr('valor_pagamento_normalizado', 'parcelas_pagamento_normalizado')

0.33081084451898507

In [7]:
itens_pedido_df = join_itens_pedido.na.drop()

montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas',
    handleInvalid='skip'
)

itens_pedido_df = montar_vetor.transform(itens_pedido_df)
itens_pedido_df.show()

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------+--------------------+---------------

In [8]:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(itens_pedido_df)

itens_pedido_df = modelo_scaler.transform(itens_pedido_df)
itens_pedido_df.show()

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+---------------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|caracteristicas_normalizado|
+--------------------+----------------

In [9]:
train_data, test_data = itens_pedido_df.randomSplit([0.7, 0.3], seed=42)

print(f'Treino: {train_data.count()} / Teste: {test_data.count()}')

Treino: 76178 / Teste: 32465


In [11]:
regressao_linear = LinearRegression(featuresCol='caracteristicas_normalizado', labelCol='valor_frete')
modelo_lr = regressao_linear.fit(train_data)

avaliar_test = modelo_lr.evaluate(test_data)
print(f'RMSE (Erro Quadrático Médio Raiz): {avaliar_test.rootMeanSquaredError}')
print(f'R2 (Coeficiente de Determinação): {avaliar_test.r2}')

RMSE (Erro Quadrático Médio Raiz): 12.29261332362075
R2 (Coeficiente de Determinação): 0.3656767551944773


In [12]:
spark.stop()